# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

#### Instalar Selenium

In [69]:
#!pip install selenium

#### Importando WebDriver do Selenium 

In [70]:
import pandas as pd #Biblioteca de análise de dados
from selenium import webdriver #Controlar navegador
from selenium.webdriver.common.keys import Keys #Usar teclado
from selenium.webdriver.common.by import By #Localizador de itens do navegador

#### Utilizando webdriver para buscar as cotações

In [71]:
#Abrindo navegador
navegador = webdriver.Chrome()
#Abrindo Site
navegador.get("https://google.com")
#Referenciando elemento de input da página pelo XPath, escrevendo e teclando 'Enter'
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
                      ).send_keys("Cotação dólar", Keys.ENTER)
#cot_dolar recebe 'data-value', que contém o valor da cotação
cot_dolar = navegador.find_element(By.XPATH, 
                                   '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
                                  ).get_attribute('data-value')
#Mostra cotação no console
print("Cotação do Dólar: ",cot_dolar)

#Abrindo Site
navegador.get("https://google.com")
#Referenciando elemento de input da página pelo XPath, escrevendo e teclando 'Enter'
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
                      ).send_keys("Cotação euro", Keys.ENTER)
#cot_euro recebe 'data-value', que contém o valor da cotação do euro
cot_euro = navegador.find_element(By.XPATH,
                                   '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
                                 ).get_attribute('data-value')
print("Cotação do Euro: ",cot_euro)

navegador.get('https://www.melhorcambio.com/ouro-hoje')
cot_ouro = navegador.find_element(By.XPATH,
                                   '//*[@id="comercial"]'
                                 ).get_attribute('value')
#cot_ouro recebe 'data-value', que contém o valor da cotação do ouro
cot_ouro = cot_ouro.replace(",",".")
print("Cotação do Ouro: ",cot_ouro)

navegador.quit()

Cotação do Dólar:  5.0116
Cotação do Euro:  5.5120984680000005
Cotação do Ouro:  324.63


### Atualizando a base de preços com as novas cotações

- Importando a base de dados

In [72]:
tabela = pd.read_excel("Produtos.xlsx")

- Atualizando os preços e o cálculo do Preço Final

In [73]:
#Atualizando cotação do dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cot_dolar)
#Atualizando cotação do euro
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cot_euro)
#Atualizando cotação do ouro
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cot_ouro)
#Atualizando preço de compra 
tabela["Preço de Compra"]= tabela["Preço Original"] * tabela["Cotação"]
#Atualizando preço de venda 
tabela["Preço de Venda"]= tabela["Preço de Compra"] * tabela["Margem"]
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.011600,5011.549884,1.40,7016.169838
1,Carro Renault,4500.00,Euro,5.512098,24804.443106,2.00,49608.886212
2,Notebook Dell,899.99,Dólar,5.011600,4510.389884,1.70,7667.662803
3,IPhone,799.00,Dólar,5.011600,4004.268400,1.70,6807.256280
4,Carro Fiat,3000.00,Euro,5.512098,16536.295404,1.90,31418.961268
5,Celular Xiaomi,480.48,Dólar,5.011600,2407.973568,2.00,4815.947136
6,Joia 20g,20.00,Ouro,324.630000,6492.600000,1.15,7466.490000


### Exportando a nova base de dados atualizada

In [74]:
tabela.to_excel("Produtos'Novo'.xlsx", index = 0)